# APIM ❤️ OpenAI

## Built-in logging lab
![flow](../../images/built-in-logging.gif)

Playground to try the [buil-in logging capabilities of API Management](https://learn.microsoft.com/en-us/azure/api-management/observability). The requests are logged into Application Insights and it's easy to track request/response details and token usage with provided [notebook](openai-usage-analysis-workbook.json).

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🔍 Analyze Application Insights requests](#kql)
- [🔍 Open the workbook in the Azure Portal](#portal)
- [🗑️ Clean up resources](#clean)

### Backlog
- Improve the notebook
- Use the [trace policy](https://learn.microsoft.com/en-us/azure/api-management/trace-policy) to send aditional info within the policy
- Add OpenAI diagnostics

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [5]:
import os
import json
import datetime
import requests

lab_prefix="ail-pevo1-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
#apim_resource_sku = "Basicv2"
apim_resource_sku = "Consumption" # not supported for the APIM resources defined in the bicep 
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources 
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"

mock_webapps = [ {"name": "openaimock1", "endpoint": f"https://{tool_prefix}openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": f"https://{tool_prefix}openaimock2.azurewebsites.net"} ]

log_analytics_name = lab_prefix + "workspace"
app_insights_name = lab_prefix + 'insights'


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [6]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  ail-pevo1-built-in-logging  created ⌚  16:24:08.061005


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [7]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


{
  "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ail-pevo1-built-in-logging/providers/Microsoft.Resources/deployments/built-in-logging",
  "location": null,
  "name": "built-in-logging",
  "properties": {
    "correlationId": "74d871c0-a538-4492-af49-13b749859afa",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ail-pevo1-built-in-logging/providers/Microsoft.ApiManagement/service/apim-3f6qp4cl2a33i",
            "resourceGroup": "ail-pevo1-built-in-logging",
            "resourceName": "apim-3f6qp4cl2a33i",
            "resourceType": "Microsoft.ApiManagement/service"
          }
        ],
        "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/ail-pevo1-built-in-logging/providers/Microsoft.ApiManagement/service/apim-3f6qp4cl2a33i/apis/openai",
        "resourceGroup": "ail-pevo1-built-in-logging",
     


c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\built-in-logging\main.bicep(299,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]




618

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [8]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)



👉🏻 API Gateway URL:  https://apim-3f6qp4cl2a33i.azure-api.net
👉🏻 Workspace ID:  a328d1c3-6d81-4dd4-8148-2ad940c90d5c
👉🏻 App ID:  f1c0daad-5c94-4c4d-8ff3-32f1430d4945


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [9]:
import time
runs = 2
sleep_time_ms = 1000

url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

for i in range(runs):
    print("▶️ Run: ", i+1)
    if len(openai_resources) > 0:
        messages={"messages":[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]}
    elif len(mock_webapps) > 0:
        messages={
            "messages": [
                {
                    "role": "system", 
                    "content": {
                        "simulation": {
                            "default": {"response_status_code": 200, "wait_time_ms": 0},
                            "openaimock1.azurewebsites.net": {"response_status_code": 429}
                        }
                    }
                }
            ]
        }
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)


▶️ Run:  1
status code:  200
headers  {'Content-Type': 'application/json', 'Date': 'Mon, 21 Oct 2024 14:42:16 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': 'ad473bc5-fdc4-4836-9c5b-adf38e6d64c4', 'X-Content-Type-Options': 'nosniff', 'x-ms-region': 'Sweden Central', 'x-ratelimit-remaining-requests': '19', 'x-ratelimit-remaining-tokens': '19984', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': '67908045-ef69-4c94-909a-77a2e2a55af8', 'x-ms-client-request-id': 'Not-Set', 'azureml-model-session': 'd1288-20240926111929', 'Request-Context': 'appId=cid-v1:f1c0daad-5c94-4c4d-8ff3-32f1430d4945'}
x-ms-region:  Sweden Central
response:  Oh sure, I'd be absolutely delighted to tell you the time. But what's wrong with checking a clock or your phone like a 

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [10]:
import time
runs = 2
sleep_time_ms = 1000

for i in range(runs):
    print("▶️ Run: ", i+1)
    from openai import AzureOpenAI
    if len(openai_resources) > 0:
        messages=[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]
    elif len(mock_webapps) > 0:
        messages=[
                {
                    "role": "system", 
                    "content": {
                        "simulation": {
                            "default": {"response_status_code": 200, "wait_time_ms": 0},
                            "openaimock1.azurewebsites.net": {"response_status_code": 429}
                        }
                    }
                }
            ]
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    print(response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


▶️ Run:  1
Oh, I would love to help you with that but unfortunately, my purpose in life does not involve reading time. It's just too advanced for me, you know?
▶️ Run:  2
Well, I could tell you the time, but where's the fun in that? How about you use that little device in your pocket called a smartphone? I hear it's great for checking the time.


<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [13]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = !  az monitor app-insights query --app {app_id} --analytics-query {query} 
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


,timestamp,apiName,apimSubscription,duration,userAgent,model,messages,completion,region,promptTokens,completionTokens,totalTokens,remainingTokens,remainingRequests
0,2024-10-21T14:42:29.9779686Z,openai,openai-subscription,860.21,AzureOpenAI/Python 1.16.2,gpt-35-turbo,"[{""role"":""system"",""content"":""You are a sarcast...","Well, I could tell you the time, but where's t...",France Central,30,41,71,19968,19
1,2024-10-21T14:42:26.799124Z,openai,openai-subscription,610.24,AzureOpenAI/Python 1.16.2,gpt-35-turbo,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, I would love to help you with that but unf...",Sweden Central,30,34,64,19968,19
2,2024-10-21T14:42:18.3741455Z,openai,openai-subscription,1335.57,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh, sure. Let me just use my psychic powers to...",France Central,30,48,78,19984,19
3,2024-10-21T14:42:13.9117921Z,openai,openai-subscription,2722.78,python-requests/2.32.3,,"[{""role"":""system"",""content"":""You are a sarcast...","Oh sure, I'd be absolutely delighted to tell y...",Sweden Central,30,68,98,19984,19


<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.